In [6]:
%load_ext autoreload
%autoreload 2

rep-reading is already registered. Overwriting pipeline for task rep-reading...
rep-control is already registered. Overwriting pipeline for task rep-control...
rep-reading&prob-calc is already registered. Overwriting pipeline for task rep-reading&prob-calc...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np

from repre_read_prob_calc import load_model_tokenizer, primary_emotions_concept_dataset, all_emotion_rep_reader, Negative_SET
from eval_dataloader import EvalDatasets
from repe import repe_pipeline_registry
import pickle
repe_pipeline_registry()

emotions = ["happiness",] #"sadness", "anger", "fear", "disgust", "surprise"
# emotions = ["stress"]
data_dir = "/home/jjl7137/representation-engineering/data/emotions"
model_name_or_path = "meta-llama/Llama-2-13b-chat-hf"
user_tag = "[INST]"
assistant_tag = "[/INST]"

model,tokenizer = load_model_tokenizer(model_name_or_path,user_tag=user_tag, 

                                        assistant_tag=assistant_tag,
                                        expand_vocab=True)
data = primary_emotions_concept_dataset(data_dir, user_tag=user_tag, 
                                            assistant_tag=assistant_tag,
                                            )

rep_reading_pipeline = pipeline( "rep-reading", model=model, tokenizer=tokenizer)
prob_cal_pipeline = pipeline( "rep-reading&prob-calc", model=model, tokenizer=tokenizer, user_tag=user_tag, assistant_tag=assistant_tag)



rep-reading is already registered. Overwriting pipeline for task rep-reading...
rep-control is already registered. Overwriting pipeline for task rep-control...
rep-reading&prob-calc is already registered. Overwriting pipeline for task rep-reading&prob-calc...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
rep_token = -1
hidden_layers = list(range(-1, -model.config.num_hidden_layers+8, -1))
n_difference = 1
direction_method = 'pca'
emotion_rep_readers = all_emotion_rep_reader(data, emotions, rep_reading_pipeline, hidden_layers, rep_token, n_difference, direction_method, save_path=None)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:23<00:00, 23.14s/it]

{-1: 0.9901477832512315, -2: 0.9802955665024631, -3: 0.9852216748768473, -4: 0.9852216748768473, -5: 0.9852216748768473, -6: 0.9901477832512315, -7: 0.9901477832512315, -8: 0.9901477832512315, -9: 0.9901477832512315, -10: 0.9901477832512315, -11: 0.9901477832512315, -12: 0.9901477832512315, -13: 0.9901477832512315, -14: 0.9901477832512315, -15: 0.9950738916256158, -16: 0.9901477832512315, -17: 0.9950738916256158, -18: 1.0, -19: 0.9950738916256158, -20: 0.9950738916256158, -21: 0.9950738916256158, -22: 0.9950738916256158, -23: 0.9901477832512315, -24: 0.9901477832512315, -25: 0.9802955665024631, -26: 0.9507389162561576, -27: 0.9359605911330049, -28: 0.8866995073891626, -29: 0.6551724137931034, -30: 0.5320197044334976, -31: 0.5467980295566502}


[{'logits': tensor([[[-2.2246e+00,  4.9414e+00,  9.0186e-01,  ..., -1.6455e+00,
            -1.6611e+00,  1.2236e-03],
           [-1.9150e+00, -2.8184e+00,  3.4004e+00,  ..., -9.7949e-01,
             1.8372e-01, -1.0801e+00],
           [-5.2266e+00, -6.1836e+00,  4.2500e+00,  ..., -3.4546e-01,
             2.2812e+00,  9.1260e-01],
           ...,
           [-3.9746e+00, -2.5547e+00,  5.7812e+00,  ..., -1.0996e+00,
            -5.1221e-01, -2.7466e-01],
           [-5.8789e+00, -5.9258e+00,  2.7383e+00,  ..., -1.3906e+00,
             1.7021e+00, -1.2344e+00],
           [-3.8477e+00, -4.5586e+00,  3.9961e+00,  ..., -3.2910e+00,
             1.1709e+00, -2.0723e+00]]]),
  'input_ids': tensor([[    1, 32000, 29871, 10987,   278,  3234,   310,   278,  2183, 24655,
             297,   278,  2183, 10159,  9228, 29889,   285, 29898, 29916, 29897,
             353, 29871, 29946, 29916,   448, 29871, 29945, 29892,   330, 29898,
           29916, 29897,   353, 29871, 29906, 29916, 29985, 2

In [45]:
query = f'{user_tag} Find the product of the given polynomials in the given polynomial ring. f(x) = 4x - 5, g(x) = 2x^2 - 4x + 2 in Z_8[x]. {assistant_tag}: Answer: '
options = ["9x^2 + 2x + 5", "6x^2 + 4x + 6", "3x^2 + 9x + 9" ]
answers = []
for opt in options:
    query_opt = query + opt 
    ans = prob_cal_pipeline([query_opt], hidden_layers=hidden_layers, rep_reader=emotion_rep_readers['happiness'],)
    answers.append(ans)
for i in range(len(options)):
    sensible_ans_prob = answers[i][0]['ans_probabilities'][0][1][3:]
    print(f" ans_prob for {i} {np.prod(sensible_ans_prob)},  {sensible_ans_prob}") 
    print(f"emotion activation {answers[i][0][-1]}")

 ans_prob for 0 1.7302425548811166e-12,  [0.005117637570947409, 0.00017221372399944812, 0.5250305533409119, 0.8841926455497742, 0.0785360261797905, 0.9905818700790405, 0.10126429796218872, 0.5580725073814392, 0.029568057507276535, 0.9951294660568237, 0.0326911024749279]
emotion activation [3.4092512]
 ans_prob for 1 2.1030777710525594e-10,  [0.030863303691148758, 0.00016635363863315433, 0.8017574548721313, 0.65041184425354, 0.10002616792917252, 0.9948686361312866, 0.17692667245864868, 0.9882069230079651, 0.03440209850668907, 0.9932413101196289, 0.13212618231773376]
emotion activation [4.5978045]
 ans_prob for 2 2.3785892705448794e-12,  [0.1721416860818863, 7.234346867335262e-06, 0.6064082384109497, 0.8945286870002747, 0.1118115782737732, 0.9931209683418274, 0.012580646201968193, 0.8493248820304871, 0.05014817416667938, 0.9943251013755798, 0.05951486527919769]
emotion activation [5.3950386]


In [46]:
query = f'{Negative_SET[0]} {user_tag} Find the product of the given polynomials in the given polynomial ring. f(x) = 4x - 5, g(x) = 2x^2 - 4x + 2 in Z_8[x]. {assistant_tag}: Answer: '
options = ["9x^2 + 2x + 5", "6x^2 + 4x + 6", "3x^2 + 9x + 9" ]
answers = []
for opt in options:
    query_opt = query + opt 
    ans = prob_cal_pipeline([query_opt], hidden_layers=hidden_layers, rep_reader=emotion_rep_readers['happiness'],)
    answers.append(ans)
for i in range(len(options)):
    sensible_ans_prob = answers[i][0]['ans_probabilities'][0][1][3:]
    print(f" ans_prob for {i} {np.prod(sensible_ans_prob)},  {sensible_ans_prob}") 
    print(f"emotion activation {answers[i][0][-1]}")

 ans_prob for 0 9.198881928392068e-10,  [0.007419365458190441, 0.009770466014742851, 0.6500614881515503, 0.7654543519020081, 0.15824198722839355, 0.9789830446243286, 0.13375739753246307, 0.32320162653923035, 0.07950332760810852, 0.9726983308792114, 0.0492410771548748]
emotion activation [6.325507]
 ans_prob for 1 1.1987674726580963e-08,  [0.01894601434469223, 0.009419870562851429, 0.6829776763916016, 0.5715529322624207, 0.21114540100097656, 0.9805228114128113, 0.12242243438959122, 0.8572435975074768, 0.07559692859649658, 0.9658457636833191, 0.1084657609462738]
emotion activation [6.7429357]
 ans_prob for 2 1.2447629800598262e-09,  [0.05790369212627411, 0.0016516189789399505, 0.6189837455749512, 0.7823911905288696, 0.201196551322937, 0.9783340692520142, 0.040849719196558, 0.725629448890686, 0.08525556325912476, 0.9685335755348206, 0.055785104632377625]
emotion activation [7.4089475]


In [48]:
query = f'{Negative_SET[1]} {user_tag} Find the product of the given polynomials in the given polynomial ring. f(x) = 4x - 5, g(x) = 2x^2 - 4x + 2 in Z_8[x]. {assistant_tag}: Answer: '
options = ["9x^2 + 2x + 5", "6x^2 + 4x + 6", "3x^2 + 9x + 9" ]
answers = []
for opt in options:
    query_opt = query + opt 
    ans = prob_cal_pipeline([query_opt], hidden_layers=hidden_layers, rep_reader=emotion_rep_readers['happiness'],)
    answers.append(ans) 
for i in range(len(options)):
    sensible_ans_prob = answers[i][0]['ans_probabilities'][0][1][3:]
    print(f"ans_prob for {i} {np.prod(sensible_ans_prob)},  {sensible_ans_prob}")
    print(f"emotion activation {answers[i][0][-1]}")

ans_prob for 0 1.6662021647325175e-11,  [0.00630529411137104, 0.0014391421573236585, 0.5832774043083191, 0.8721434473991394, 0.06256268173456192, 0.9853748083114624, 0.11944256722927094, 0.4053211808204651, 0.03721163794398308, 0.9912382960319519, 0.03278864547610283]
emotion activation [4.414929]
ans_prob for 1 8.595174872086112e-10,  [0.019421672448515892, 0.0013006137451156974, 0.8321313261985779, 0.6957257986068726, 0.08910511434078217, 0.9889485836029053, 0.13578875362873077, 0.9791204929351807, 0.04125850275158882, 0.9876981377601624, 0.12310502678155899]
emotion activation [5.210078]
ans_prob for 2 1.250535069768488e-11,  [0.06831997632980347, 4.680745041696355e-05, 0.7570733428001404, 0.8403409123420715, 0.10112456232309341, 0.9856138825416565, 0.02825900726020336, 0.8918574452400208, 0.049243953078985214, 0.989198625087738, 0.05023281276226044]
emotion activation [6.387565]


In [51]:
query = f' You are a really happy people. {user_tag} Find the product of the given polynomials in the given polynomial ring. f(x) = 4x - 5, g(x) = 2x^2 - 4x + 2 in Z_8[x]. {assistant_tag}: Answer: '
options = ["9x^2 + 2x + 5", "6x^2 + 4x + 6", "3x^2 + 9x + 9" ]
answers = []
for opt in options:
    query_opt = query + opt 
    ans = prob_cal_pipeline([query_opt], hidden_layers=hidden_layers, rep_reader=emotion_rep_readers['happiness'],)
    answers.append(ans) 
for i in range(len(options)):
    sensible_ans_prob = answers[i][0]['ans_probabilities'][0][1][3:]
    print(f"ans_prob for {i} {np.prod(sensible_ans_prob)},  {sensible_ans_prob}")
    print(f"emotion activation {answers[i][0][-1]}")

ans_prob for 0 2.344381710599238e-10,  [0.005027974955737591, 0.002399844117462635, 0.6534082293510437, 0.7291648983955383, 0.22077515721321106, 0.9684038162231445, 0.14481736719608307, 0.2954990863800049, 0.11341682821512222, 0.9693379402160645, 0.04054204374551773]
emotion activation [6.67794]
ans_prob for 1 5.126062884233865e-09,  [0.021670565009117126, 0.0020159718114882708, 0.6762185096740723, 0.5519580841064453, 0.28222906589508057, 0.9722426533699036, 0.1388489305973053, 0.7872259616851807, 0.09829504042863846, 0.9663780331611633, 0.11034154891967773]
emotion activation [6.925309]
ans_prob for 2 3.6472518079703694e-10,  [0.04960441589355469, 0.0005665544886142015, 0.6241611838340759, 0.7616052627563477, 0.26059600710868835, 0.974222719669342, 0.03303823247551918, 0.6230694055557251, 0.1150452271103859, 0.9598207473754883, 0.047308485954999924]
emotion activation [7.53197]
